In [1]:
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/dataset_emotion_preprocessed_augmented.pickle', 'rb') as f:
   dataset_emotion = pickle.load(f)

In [4]:
dataset_emotion = dataset_emotion.sample(frac=1)

In [5]:
dataset_emotion = dataset_emotion[dataset_emotion['emotion'] != 'surprise']
len(dataset_emotion)

152440

In [6]:
dataset_emotion["emotion"].value_counts()

happiness    35371
sadness      33208
worry        31944
neutral      26746
love         25171
Name: emotion, dtype: int64

In [7]:
tokenizer = Tokenizer(num_words=5000, split=' ')

tokenizer.fit_on_texts(dataset_emotion['processed'].values)

X = tokenizer.texts_to_sequences(dataset_emotion['processed'])

X = pad_sequences(X)

In [8]:
files = open("tokenizer_emotion.pickle", "wb")
pickle.dump(tokenizer, files)

In [9]:
embed_dim = 300
lstm_out = 128

model = Sequential()
model.add(Embedding(5000, 300, input_length = X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 300)           1500000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 1,720,293
Trainable params: 1,720,293
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
Y = pd.get_dummies(dataset_emotion['emotion']).values
print(Y[:20])
print(dataset_emotion['emotion'][:20])

[[0 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 1 0]]
131095      neutral
141691    happiness
144100    happiness
44146     happiness
122839      neutral
92244          love
148989    happiness
83523          love
22997     happiness
17744       sadness
85929          love
127021      neutral
136490      neutral
151664    happiness
151302    happiness
85707          love
75628         worry
31984         worry
144093    happiness
112896      sadness
Name: emotion, dtype: object


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [12]:
batch_size = 64
model.fit(X_train, y_train, epochs = 35, batch_size=batch_size, verbose = 1)

Epoch 1/35
1787/1787 [==============================] - 446s 235ms/step - loss: 0.6287 - accuracy: 0.7603
Epoch 2/35
1787/1787 [==============================] - 428s 240ms/step - loss: 0.3846 - accuracy: 0.8493
Epoch 3/35
1787/1787 [==============================] - 444s 248ms/step - loss: 0.3648 - accuracy: 0.8574
Epoch 4/35
1787/1787 [==============================] - 463s 259ms/step - loss: 0.3469 - accuracy: 0.8651
Epoch 5/35
1787/1787 [==============================] - 456s 255ms/step - loss: 0.3366 - accuracy: 0.8704
Epoch 6/35
1787/1787 [==============================] - 480s 269ms/step - loss: 0.3235 - accuracy: 0.8760
Epoch 7/35
1787/1787 [==============================] - 488s 273ms/step - loss: 0.3132 - accuracy: 0.8808
Epoch 8/35
1787/1787 [==============================] - 486s 272ms/step - loss: 0.2993 - accuracy: 0.8865
Epoch 9/35
1787/1787 [==============================] - 463s 259ms/step - loss: 0.2894 - accuracy: 0.8893
Epoch 10/35
1787/1787 [=======================

In [13]:
model.save('/content/drive/MyDrive/emotions_model_lstm_6')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/emotions_model_lstm_6/assets


In [14]:
score, accuracy = model.evaluate(X_test, y_test, verbose=1)

1191/1191 [==============================] - 34s 27ms/step - loss: 0.8589 - accuracy: 0.8251
